In [1]:
import numpy as np
from scipy.stats import t as tdist, ttest_1samp, ttest_ind

np.random.seed(2)

In [2]:
def check_stats_significance(p_value, threshold=0.05):
    if p_value < threshold:
        return f'{p_value:.4f} < {threshold:.2f}: Reject the null hypothesis H0; the result statistically significant'
    else:
        return f'{p_value:.4f} >= {threshold:.2f}: Failed to reject the null hypothesis H0; the result not statistically significant'

# 1. 1-sample t-test
## 1.1 1-sample, 2-sided test

- $H_0$: $\mu = \mu_0$
- $H_1$: $\mu \neq \mu_0$

In [3]:
# data
N = 100
mu = 0.2
sigma = 1

x = np.random.randn(N)*sigma + mu

# Compare to mu_0 = 0.2
mu_0 = 0.2

#### API

In [4]:
test_statistic, p_value = ttest_1samp(x, popmean=mu_0)

print(p_value)
print(check_stats_significance(p_value))

0.3219937839012001
0.3220 >= 0.05: Failed to reject the null hypothesis H0; the result not statistically significant


#### Implementation

In [5]:
mu_hat = x.mean()
sigma_hat = x.std(ddof=1)

t = (mu_hat - mu_0) / (sigma_hat / np.sqrt(N))
p_right = 1 - tdist.cdf(np.abs(t), df=N-1)
p_left = tdist.cdf(-np.abs(t), df=N-1)
p_value = p_right + p_left

print(t, p_value)
print(check_stats_significance(p_value))

-0.9953477389335027 0.32199378390120015
0.3220 >= 0.05: Failed to reject the null hypothesis H0; the result not statistically significant


## 1.2 1-sample, 1-sided test
- $H_0$: $\mu \leq \mu_0$
- $H_1$: $\mu > \mu_0$

In [6]:
# data
N = 100
mu = 0.2
sigma = 1

x = np.random.randn(N)*sigma + mu

# Compare to mu_0 = 0
mu_0 = 0

#### API

In [7]:
test_statistic, res = ttest_1samp(x, popmean=mu_0)
p_value = res/2

print(p_value)
print(check_stats_significance(p_value))

0.0020182811405012355
0.0020 < 0.05: Reject the null hypothesis H0; the result statistically significant


#### Implementation

In [8]:
mu_hat = x.mean()
sigma_hat = x.std(ddof=1)

t = (mu_hat - mu_0) / (sigma_hat / np.sqrt(N))
p_value = 1 - tdist.cdf(t, df=N-1)

print(t, p_value)
print(check_stats_significance(p_value))

2.944060629425759 0.002018281140501177
0.0020 < 0.05: Reject the null hypothesis H0; the result statistically significant


# 2. 2-sample z-test
## 2.1 2-sample, 2-sided test

- $H_0$: $\mu_1 = \mu_2$
- $H_1$: $\mu_1 \neq \mu_2$

In [9]:
def ttest_2samp_2sides(x_1, x_2, equal_var=False):
    N_1 = len(x_1)
    N_2 = len(x_2)

    mu_hat_1 = x_1.mean()
    mu_hat_2 = x_2.mean()

    sq_sigma_hat_1 = x_1.var(ddof=1)
    sq_sigma_hat_2 = x_2.var(ddof=1)

    if N_1 == N_2 and equal_var == True:
        # Case 1
        sq_sp = (sq_sigma_hat_1 + sq_sigma_hat_2) / 2
        t = (mu_hat_1 - mu_hat_2) / (np.sqrt(sq_sp) * np.sqrt(2/N_1))
        v = 2*N_1 - 2 
    elif N_1 != N_2 and equal_var == True:
        # Case 2 
        sq_sp = ((N_1 - 1)*sq_sigma_hat_1 + (N_2 - 1)*sq_sigma_hat_2) / (N_1 + N_2 - 2)
        t = (mu_hat_1 - mu_hat_2) / (np.sqrt(sq_sp) * np.sqrt(1/N_1 + 1/N_2))
        v = N_1 + N_2 - 2 
    else:
        # Case 3
        t = (mu_hat_1 - mu_hat_2) / (np.sqrt(sq_sigma_hat_1/N_1 + sq_sigma_hat_2/N_2))
        v = (sq_sigma_hat_1/N_1 + sq_sigma_hat_2/N_2)**2 / ((sq_sigma_hat_1/N_1)**2 / (N_1 - 1) + (sq_sigma_hat_2/N_2)**2 / (N_2 - 1))

    p_right = 1 - tdist.cdf(np.abs(t), df=v)
    p_left = tdist.cdf(-np.abs(t), df=v)
    p_value = p_right + p_left
    return t, p_value

#### dataset 1

In [10]:
# case 1: N1 = N2, same variance
N = 100
sigma = 1

mu_1 = 0
x_1 = np.random.randn(N)*sigma + mu_1

mu_2 = 0.5
x_2 = np.random.randn(N)*sigma + mu_2

In [11]:
#### API
test_statistic, p_value = ttest_ind(x_1, x_2, equal_var=True)

print(p_value)
print(check_stats_significance(p_value))

2.712072484183902e-10
0.0000 < 0.05: Reject the null hypothesis H0; the result statistically significant


In [12]:
#### Implementation
t, p_value = ttest_2samp_2sides(x_1, x_2, equal_var=True)

print(t, p_value)
print(check_stats_significance(p_value))

-6.655152478367741 2.712072636376339e-10
0.0000 < 0.05: Reject the null hypothesis H0; the result statistically significant


#### dataset 2

In [13]:
# case 2: N1 != N2, same variance
sigma = 1

N_1 = 100
mu_1 = 0
x_1 = np.random.randn(N_1)*sigma + mu_1

N_2 = 50
mu_2 = 0.5
x_2 = np.random.randn(N_2)*sigma + mu_2

In [14]:
#### API
test_statistic, p_value = ttest_ind(x_1, x_2, equal_var=True)

print(p_value)
print(check_stats_significance(p_value))

0.025752977958207125
0.0258 < 0.05: Reject the null hypothesis H0; the result statistically significant


In [15]:
#### Implementation
t, p_value = ttest_2samp_2sides(x_1, x_2, equal_var=True)

print(t, p_value)
print(check_stats_significance(p_value))

-2.252633951961539 0.0257529779582071
0.0258 < 0.05: Reject the null hypothesis H0; the result statistically significant


#### dataset 3

In [16]:
# case 3: N1 != N2, different variance (Welch's t-test)
N_1 = 100
mu_1 = 0
sigma_1 = 1
x1 = np.random.randn(N_1)*sigma_1 + mu_1

N_2 = 50
mu_2 = 0.5
sigma_2 = 0.5
x2 = np.random.randn(N_2)*sigma_2 + mu_2

In [17]:
#### API
test_statistic, p_value = ttest_ind(x_1, x_2, equal_var=False)

print(p_value)
print(check_stats_significance(p_value))

0.03665736431958793
0.0367 < 0.05: Reject the null hypothesis H0; the result statistically significant


In [18]:
#### Implementation
t, p_value = ttest_2samp_2sides(x_1, x_2, equal_var=False)

print(t, p_value)
print(check_stats_significance(p_value))

-2.123358319781523 0.036657364319587965
0.0367 < 0.05: Reject the null hypothesis H0; the result statistically significant


## 2.2 2-sample, 1-sided test

- $H_0$: $\mu_1 \leq \mu_2$
- $H_1$: $\mu_1 > \mu_2$

In [19]:
def ttest_2samp_1sides_greater(x_1, x_2, equal_var=False):
    N_1 = len(x_1)
    N_2 = len(x_2)

    mu_hat_1 = x_1.mean()
    mu_hat_2 = x_2.mean()

    sq_sigma_hat_1 = x_1.var(ddof=1)
    sq_sigma_hat_2 = x_2.var(ddof=1)

    if N_1 == N_2 and equal_var == True:
        # Case 1
        sq_sp = (sq_sigma_hat_1 + sq_sigma_hat_2) / 2
        t = (mu_hat_1 - mu_hat_2) / (np.sqrt(sq_sp) * np.sqrt(2/N_1))
        v = 2*N_1 - 2 
    elif N_1 != N_2 and equal_var == True:
        # Case 2 
        sq_sp = ((N_1 - 1)*sq_sigma_hat_1 + (N_2 - 1)*sq_sigma_hat_2) / (N_1 + N_2 - 2)
        t = (mu_hat_1 - mu_hat_2) / (np.sqrt(sq_sp) * np.sqrt(1/N_1 + 1/N_2))
        v = N_1 + N_2 - 2 
    else:
        # Case 3
        t = (mu_hat_1 - mu_hat_2) / (np.sqrt(sq_sigma_hat_1/N_1 + sq_sigma_hat_2/N_2))
        v = (sq_sigma_hat_1/N_1 + sq_sigma_hat_2/N_2)**2 / ((sq_sigma_hat_1/N_1)**2 / (N_1 - 1) + (sq_sigma_hat_2/N_2)**2 / (N_2 - 1))

    p_value = 1 - tdist.cdf(t, df=v)
    return t, p_value

#### dataset 1

In [20]:
# case 1: N1 = N2, same variance
N = 100
sigma = 1

mu_1 = 0
x_1 = np.random.randn(N)*sigma + mu_1

mu_2 = 0.5
x_2 = np.random.randn(N)*sigma + mu_2

In [21]:
#### Implementation
t, p_value = ttest_2samp_1sides_greater(x_1, x_2, equal_var=True)

print(t, p_value)
print(check_stats_significance(p_value))

-4.753803318160059 0.9999980828482775
1.0000 >= 0.05: Failed to reject the null hypothesis H0; the result not statistically significant


#### dataset 2

In [22]:
# case 2: N1 != N2, same variance
sigma = 1

N_1 = 100
mu_1 = 1
x_1 = np.random.randn(N_1)*sigma + mu_1

N_2 = 50
mu_2 = 0.5
x_2 = np.random.randn(N_2)*sigma + mu_2

In [23]:
#### Implementation
t, p_value = ttest_2samp_1sides_greater(x_1, x_2, equal_var=True)

print(t, p_value)
print(check_stats_significance(p_value))

2.2490691097263467 0.012992148009159044
0.0130 < 0.05: Reject the null hypothesis H0; the result statistically significant


#### dataset 3

In [24]:
# case 3: N1 != N2, different variance (Welch's t-test)
N_1 = 100
mu_1 = 0.5
sigma_1 = 0.01
x1 = np.random.randn(N_1)*sigma_1 + mu_1

N_2 = 50
mu_2 = 0.4
sigma_2 = 0.05
x2 = np.random.randn(N_2)*sigma_2 + mu_2

In [25]:
#### Implementation
t, p_value = ttest_2samp_1sides_greater(x_1, x_2, equal_var=False)

print(t, p_value)
print(check_stats_significance(p_value))

2.244458912306332 0.013530871610873296
0.0135 < 0.05: Reject the null hypothesis H0; the result statistically significant
